# Background

In [1]:
import platform
display(platform.system())
import os
file_download_link = 'https://data.ca.gov/dataset/3f08b68e-1d1a-4ba4-a07d-1ec3392ed191/resource/78a9d6ee-ec9a-4c25-ae34-5bac44010cb2/download/qcew_2016-2019.csv'
if os.name == 'nt':
    print('Please download your dataset here:', file_download_link)
else:
    # If on another OS, use wget to download the CSV file directly
    !wget -O qcew_2016-2019.csv "$file_download_link" -o /dev/null
    print("File downloaded successfully as qcew_2016-2019.csv")

'Linux'

File downloaded successfully as qcew_2016-2019.csv


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
!{sys.executable} -m pip install imbalanced-learn delayed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 11.2 MB/s eta 0:00:00


In [4]:
import pandas as pd
# If put the data(.csv) under the same folder, you could use
df = pd.read_csv('./qcew_2016-2019.csv')
print(df.shape)
df.head()

(1009955, 15)


,Area Type,Area Name,Year,Quarter,Ownership,NAICS Level,NAICS Code,Industry Name,Establishments,Average Monthly Employment,1st Month Emp,2nd Month Emp,3rd Month Emp,Total Wages (All Workers),Average Weekly Wages
0,County,Alameda County,2016,1st Qtr,Federal Government,2,1023,Financial Activities,1,10,10,10,10,359690.0,2767.0
1,County,Alameda County,2016,1st Qtr,Federal Government,2,1028,Public Administration,72,5174,5193,5143,5188,136068970.0,2023.0
2,County,Alameda County,2016,1st Qtr,Private,5,11121,Vegetable and Melon Farming,7,29,31,27,29,210934.0,560.0
3,County,Alameda County,2016,1st Qtr,Private,4,1114,Greenhouse and Nursery Production,7,52,61,64,31,748616.0,1107.0
4,County,Alameda County,2016,1st Qtr,Private,5,11142,Nursery and Floriculture Production,7,52,61,64,31,748616.0,1107.0


In [5]:
#Lots of data points, maybe we can drop the older years
print("Number of datapoints from 2018-2019: ", df.loc[df['Year'] == 2019].shape[0] + df.loc[df['Year'] == 2018].shape[0])

Number of datapoints from 2018-2019:  506913


In [6]:
#500K datapoints should be sufficient, and it should also be more accurate since its newer data
df = df[df['Year'] != 2016]
df = df[df['Year'] != 2017]
print(df.shape)

(506913, 15)


In [7]:
df.columns

Index(['Area Type', 'Area Name', 'Year', 'Quarter', 'Ownership', 'NAICS Level',
       'NAICS Code', 'Industry Name', 'Establishments',
       'Average Monthly Employment', '1st Month Emp', '2nd Month Emp',
       '3rd Month Emp', 'Total Wages (All Workers)', 'Average Weekly Wages'],
      dtype='object')

In [8]:
#Since the NAICS code just represents the industry, and we already have indsutry name, lets drop NAICS codes and NAICS levels
df.drop(['NAICS Level','NAICS Code'],axis=1,inplace=True)
df.head()

<ipython-input-8-dc909aa15fe1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['NAICS Level','NAICS Code'],axis=1,inplace=True)


,Area Type,Area Name,Year,Quarter,Ownership,Industry Name,Establishments,Average Monthly Employment,1st Month Emp,2nd Month Emp,3rd Month Emp,Total Wages (All Workers),Average Weekly Wages
147,County,Marin County,2019,Annual,Private,Used Car Dealers,6,46,0,0,0,3170232.0,1340.0
148,County,Marin County,2019,Annual,Private,Department Stores,10,819,0,0,0,28355846.0,666.0
149,County,Marin County,2019,Annual,Private,Travel Agencies,19,189,0,0,0,10466322.0,1065.0
150,County,Modoc County,2019,Annual,Private,Logging,4,19,0,0,0,1008116.0,1048.0
151,County,Modoc County,2019,Annual,Private,Logging,4,19,0,0,0,1008116.0,1048.0


In [9]:
#We don't need to know the total wages for all of the workers, who owns the business, or the individual months of employement
df.drop(['Ownership','1st Month Emp','2nd Month Emp','3rd Month Emp','Total Wages (All Workers)'],axis=1,inplace=True)
df.head()

,Area Type,Area Name,Year,Quarter,Industry Name,Establishments,Average Monthly Employment,Average Weekly Wages
147,County,Marin County,2019,Annual,Used Car Dealers,6,46,1340.0
148,County,Marin County,2019,Annual,Department Stores,10,819,666.0
149,County,Marin County,2019,Annual,Travel Agencies,19,189,1065.0
150,County,Modoc County,2019,Annual,Logging,4,19,1048.0
151,County,Modoc County,2019,Annual,Logging,4,19,1048.0


In [10]:
#Checking what Area Types we have
df['Area Type'].unique()

array(['County', 'California - Statewide', 'United States'], dtype=object)

In [11]:
#We want to focus on the county level, lets ensure there are enough datapoints to drop the rest
print(df.loc[df['Area Type'] == 'County'].shape)
df = df[df['Area Type'] == 'County']

(441540, 8)


In [12]:
#Since there may be overlap between Quarters or the Annual datapoints, lets only use Annual
print(df.loc[df['Quarter'] == 'Annual'].shape)
df = df[df['Quarter'] == 'Annual']

(89255, 8)


In [13]:
#Now we can drop 'Area Type' and 'Quarter'
df.drop(['Area Type','Quarter'],axis=1,inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

<ipython-input-13-156b57d83d99>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Area Type','Quarter'],axis=1,inplace=True)


,Area Name,Year,Industry Name,Establishments,Average Monthly Employment,Average Weekly Wages
0,Marin County,2019,Used Car Dealers,6,46,1340.0
1,Marin County,2019,Department Stores,10,819,666.0
2,Marin County,2019,Travel Agencies,19,189,1065.0
3,Modoc County,2019,Logging,4,19,1048.0
4,Modoc County,2019,Logging,4,19,1048.0


In [14]:
df.shape

(89255, 6)

In [15]:
#Check for missing values
df.isnull().sum()

,0
Area Name,0
Year,0
Industry Name,0
Establishments,0
Average Monthly Employment,0
Average Weekly Wages,0


In [16]:
#Drop outliers using IQR
def dropOutliers(df, colName):
    [Q1, Q2, Q3] = df[colName].quantile([0.25, 0.5, 0.75])
    IQR = Q3 - Q1
    return df[(df[colName] >= (Q1 - (1.5 * IQR))) & (df[colName] <= (Q3 + (1.5 * IQR)))]
df = dropOutliers(df, 'Establishments')
df = dropOutliers(df, 'Average Monthly Employment')
df = dropOutliers(df, 'Average Weekly Wages')
print(df.shape)

(65689, 6)


In [55]:
#Splitting the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Area Name'], axis=1), df['Area Name'], test_size=0.2, random_state=21)

In [56]:
#Check the shapes
print(X_train.shape)
print(X_test.shape)

(52551, 5)
(13138, 5)


In [37]:
pip install category_encoders

In [57]:
#Import and fit the encoder
from category_encoders import BinaryEncoder
encoder = BinaryEncoder()
encoder.fit(X_train['Industry Name'])

BinaryEncoder(cols=['Industry Name'],
              mapping=[{'col': 'Industry Name',
                        'mapping':        Industry Name_0  Industry Name_1  Industry Name_2  Industry Name_3  \
 1                   0                0                0                0   
 2                   0                0                0                0   
 3                   0                0                0                0   
 4                   0                0                0                0   
 5                   0                0                0                0   
...                ...              ...              ...              ...   
 1436                1                0                1                1   
 1437                1                0                1                1   
 1438                1                0                1                1   
-1                   0                0                0                0   
-2                   0                0                0                0   

       Industry Name_4  Industry Name_5  Industry Name_6  Industry Name_7  \
 1                   0                0                0                0   
 2                   0                0                0                0   
 3                   0                0                0                0   
 4                   0                0                0                0   
 5                   0                0                0                0   
...                ...              ...              ...              ...   
 1436                0                0                1                1   
 1437                0                0                1                1   
 1438                0                0                1                1   
-1                   0                0                0                0   
-2                   0                0                0                0   

       Industry Name_8  Industry Name_9  Industry Name_10  
 1                   0                0                 1  
 2                   0                1                 0  
 3                   0                1                 1  
 4                   1                0                 0  
 5                   1                0                 1  
...                ...              ...               ...  
 1436                1                0                 0  
 1437                1                0                 1  
 1438                1                1                 0  
-1                   0                0                 0  
-2                   0                0                 0  

[1440 rows x 11 columns]}])

In [58]:
#Fit the encoder
train_industry_encoded = encoder.transform(X_train['Industry Name'])
X_train_encoded = X_train.drop(columns=['Industry Name'])
X_train_encoded = pd.concat([X_train, train_industry_encoded], axis=1)
X_train_encoded = X_train_encoded.reset_index(drop=True)

test_industry_encoded = encoder.transform(X_test['Industry Name'])
X_test_encoded = X_test.drop(columns=['Industry Name'])
X_test_encoded = pd.concat([X_test, test_industry_encoded], axis=1)
X_test_encoded = X_test_encoded.reset_index(drop=True)

In [59]:
X_train_encoded.head()

,Year,Industry Name,Establishments,Average Monthly Employment,Average Weekly Wages,Industry Name_0,Industry Name_1,Industry Name_2,Industry Name_3,Industry Name_4,Industry Name_5,Industry Name_6,Industry Name_7,Industry Name_8,Industry Name_9,Industry Name_10
0,2019,Support Activities for Mining,19,314,1817.0,0,0,0,0,0,0,0,0,0,0,1
1,2019,"Truck, Trailer and RV Rental and Leasing",54,877,1221.0,0,0,0,0,0,0,0,0,0,1,0
2,2019,Used Merchandise Stores,24,315,487.0,0,0,0,0,0,0,0,0,0,1,1
3,2019,Other Nonresidential Finishing Contrs,26,238,1031.0,0,0,0,0,0,0,0,0,1,0,0
4,2018,RV Parks and Recreational Camps,11,70,498.0,0,0,0,0,0,0,0,0,1,0,1
